# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
!pip install xlsxwriter

In [2]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np

import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
from secret_key import IEX_CLOUD_API_TOKEN
print(IEX_CLOUD_API_TOKEN)

sk_f06cf1d7484d4874af733baf46971902


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPl'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data = data[0]
print(data['symbol'])

AAPL


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = data['latestPrice']
marketCap = data['marketCap']
companyName = data['companyName']
print(price)
print(marketCap)
print(companyName)

193.055
3036504564610
Apple Inc


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = ['Ticker', 'Company Name', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Company Name,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
# final_dataframe.append(
#     pd.Series(
#     [
#         symbol,
#         price,
#         marketCap,
#         'N/A'
#     ],
#     index = my_columns),
    
#     ignore_index=True
# )

In [8]:
new_data = pd.Series(
    [symbol, companyName, price, marketCap, 'N/A'],
    index=my_columns
)

final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)

final_dataframe

,Ticker,Company Name,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPl,Apple Inc,193.055,3036504564610,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token=sk_f06cf1d7484d4874af733baf46971902'
    data = requests.get(api_url).json()
    data = data[0]

    new_data = pd.Series(
    [stock, 
     data['latestPrice'], 
     data['marketCap'], 
        'N/A'],
    index=my_columns
    )

    final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
    
    
    
    
    
    
    

In [20]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,119.3,35238335087,N/A
1,AAL,119.3,35238335087,N/A
2,AAP,119.3,35238335087,N/A
3,AAPL,119.3,35238335087,N/A
4,ABBV,119.3,35238335087,N/A
...,...,...,...,...
500,YUM,135.15,37853755752,N/A
501,ZBH,135.15,37853755752,N/A
502,ZBRA,135.15,37853755752,N/A
503,ZION,135.15,37853755752,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def batch(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
# symbol_groups = list(batch(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])
    
# final_dataframe = pd.DataFrame (columns = my_columns)

# for symbol_string in symbol_strings:
#     print(symbol_string)
    
#     batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token=sk_f06cf1d7484d4874af733baf46971902'
# #     print(batch_api_call_url)
#     data = requests.get(batch_api_call_url).json()
#     print(type(data))
#     print(data)
    
#     counter = 0
#     for symbol in symbol_string.split(','):
#         final_dataframe = final_dataframe. append(
#             pd.Series ([ symbol,
#                         data[counter]['companyName'],
#                          data[counter]['latestPrice'],
#                          data[counter]['marketCap'],
#                          'N/A'],index = my_columns), ignore_index=True)
#         counter +=1
        
# final_dataframe
    

In [10]:
symbol_groups = list(batch(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
    
final_dataframe = pd.DataFrame (columns = my_columns)

for symbol_string in symbol_strings:
    print(symbol_string)
    
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token=sk_f06cf1d7484d4874af733baf46971902'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    print(type(data))
    print(data)
    
    counter = 0
    for symbol in symbol_string.split(','):
        new_data = pd.Series(
            [symbol,
             data[counter]['companyName'],
             data[counter]['latestPrice'], 
             data[counter]['marketCap'], 
             'N/A'], index=my_columns)
        
        final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
        
        counter +=1
        
final_dataframe

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
<class 'list'>
[{'avgTotalVolume': 1666255, 'calculationPrice': 'tops', 'change': 1.99, 'changePercent': 0.01664, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': 'IEX real time price', 'highTime': 1689777122745, 'iexAskPrice': 122.76, 'iexAskSize': 122, 'iexBidPrice': 121.22, 'iexBidSize': 101, 'iexClose': 121.55, 'iexC

,Ticker,Company Name,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,Agilent Technologies Inc.,121.55,35902930678,N/A
1,AAL,American Airlines Group Inc,18.655,12179849500,N/A
2,AAP,Advance Auto Parts Inc,71.7,4262124188,N/A
3,AAPL,Apple Inc,193.13,3037684217260,N/A
4,ABBV,Abbvie Inc,137.43,242466330722,N/A
...,...,...,...,...,...
500,YUM,Yum Brands Inc.,135.29,37892967930,N/A
501,ZBH,Zimmer Biomet Holdings Inc,139.96,29191357129,N/A
502,ZBRA,Zebra Technologies Corp. - Class A,314.69,16184613065,N/A
503,ZION,Zions Bancorporation N.A,34.02,5038385848,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
portfolio_size = input('Enter the value of your portfolio:')

try:
        val = float(portfolio_size)

except ValueError:
        print("That is not a valid entry! \nPlease ")
        portfolio_size = input('Enter the value of your portfolio:')

In [12]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
    
final_dataframe    

,Ticker,Company Name,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,Agilent Technologies Inc.,121.55,35902930678,162
1,AAL,American Airlines Group Inc,18.655,12179849500,1061
2,AAP,Advance Auto Parts Inc,71.7,4262124188,276
3,AAPL,Apple Inc,193.13,3037684217260,102
4,ABBV,Abbvie Inc,137.43,242466330722,144
...,...,...,...,...,...
500,YUM,Yum Brands Inc.,135.29,37892967930,146
501,ZBH,Zimmer Biomet Holdings Inc,139.96,29191357129,141
502,ZBRA,Zebra Technologies Corp. - Class A,314.69,16184613065,62
503,ZION,Zions Bancorporation N.A,34.02,5038385848,582


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [13]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_format = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [18]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format) 
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format) 
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format) 
# writer.sheets['Recommended Trades'].set_column('E:E', 18, string_format) 
# writer.save()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Company Name', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('E1', 'Ticker', string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [16]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Company Name', string_format],
    'C': ['Stock Price', dollar_format],
    'D': ['Market Capitalization', dollar_format],
    'E': ['Number of Shares to Buy', integer_format]}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])                                    

## Saving Our Excel Output

Saving our Excel file is very easy:

In [17]:
writer.close ()